# ES → BBF Salesforce Account Migration

This notebook migrates Account records from ES Salesforce to BBF Salesforce.

## Central Policy (Order-Driven Migration)
**Only migrates Accounts that are linked to BANs marked for migration (`BBF_Ban__c = true`).**

This ensures we only migrate Accounts that have qualifying Orders:

**Include Orders where:**
- Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')

**Exclude Orders where:**
- Project_Group__c LIKE '%PA MARKET DECOM%'

All other Project_Group__c values (including NULL or any other project) are **included**.

The `BBF_Ban__c` flag is set by either:
- **Production:** `es_to_bbf_ban_creation_v12.ipynb` (creates -BBF BANs)
- **UAT Testing:** `00_uat_ban_prep.ipynb` (marks existing BANs)

## Process Overview
1. Connect to both ES (source) and BBF (target) Salesforce orgs
2. Query BANs with `BBF_Ban__c = true` to get Account IDs
3. Query Accounts linked to those BANs (not yet migrated)
4. Transform ES Accounts for BBF schema
5. Insert Accounts to BBF Salesforce
6. Update ES Accounts with `BBF_New_Id__c` = BBF Account.Id
7. Output results to Excel with color-coded status

## Field Tracking Strategy
**In BBF:** `ES_Legacy_ID__c` stores original ES Account ID

**In ES:** `BBF_New_Id__c` stores new BBF Account ID

## Safety
- `TEST_MODE = True` by default (limits to 10 Accounts)
- `FILTER_BY_BBF_BAN = True` by default (only migrates Accounts linked to BBF_Ban__c = true BANs)
- All inserts use bulk API for performance
- Verification queries after migration
- Excel output shows all successes and failures

In [1]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
import dotenv
import os

dotenv.load_dotenv()

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Imports successful")

Python: /home/vjl2dev/miniconda3/envs/kairos-gpu/bin/python
Pandas: 2.3.3
✅ Imports successful


In [2]:
# === CONFIGURATION ===
import sys
sys.path.insert(0, '..')  # Add parent directory to path for credentials module

from credentials import get_es_credentials, get_bbf_credentials

# Choose ES environment: 'uat' for sandbox, 'prod' for production
ES_ENVIRONMENT = 'uat'  # Change to 'prod' for production migration

# Get credentials from .env file
es_creds = get_es_credentials(ES_ENVIRONMENT)
bbf_creds = get_bbf_credentials()

# Expose for compatibility with existing code
ES_USERNAME = es_creds['username']
ES_PASSWORD = es_creds['password']
ES_TOKEN = es_creds['security_token']
ES_DOMAIN = es_creds['domain']

BBF_USERNAME = bbf_creds['username']
BBF_PASSWORD = bbf_creds['password']
BBF_TOKEN = bbf_creds['security_token']
BBF_DOMAIN = bbf_creds['domain']

# Migration Options
TEST_MODE = False  # Set to False to migrate ALL accounts
TEST_LIMIT = 10  # Only used when TEST_MODE = True

# Account Owner - Set all migrated accounts to this user
# Note: We'll use a service account user that exists in BBF
OWNER_ID = "005Ea00000ZOGFZIA5"  # "Everstream Legacy" user in BBF

# Output Configuration
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"es_bbf_account_migration_{timestamp}.xlsx"

print("Configuration loaded from .env file")
print(f"   ES Environment: {ES_ENVIRONMENT}")
print(f"   ES Username: {ES_USERNAME}")
print(f"   BBF Username: {BBF_USERNAME}")
print(f"   TEST_MODE: {TEST_MODE}")
print(f"   Owner ID: {OWNER_ID}")
print(f"   Output: {output_file}")
print("\nNote: Bulk API automatically handles batching (200 records/batch)")

Configuration loaded from .env file
   ES Environment: uat
   ES Username: sfdcapi@everstream.net.uat
   BBF Username: vlettau@everstream.net
   TEST_MODE: False
   Owner ID: 005Ea00000ZOGFZIA5
   Output: es_bbf_account_migration_20260120_134341.xlsx

Note: Bulk API automatically handles batching (200 records/batch)


In [3]:
# === CONNECT TO SALESFORCE ORGS ===

print("=" * 80)
print("CONNECTING TO SALESFORCE ORGS")
print("=" * 80)

# Connect to ES (source)
print("\n🔌 Connecting to ES (source)...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

# Connect to BBF (target)
print("\n🔌 Connecting to BBF (target)...")
bbf_sf = Salesforce(
    username=BBF_USERNAME,
    password=BBF_PASSWORD,
    security_token=BBF_TOKEN,
    domain=BBF_DOMAIN,
)
print(f"✅ Connected to BBF: {bbf_sf.sf_instance}")

CONNECTING TO SALESFORCE ORGS

🔌 Connecting to ES (source)...
✅ Connected to ES: everstream--uat.sandbox.my.salesforce.com

🔌 Connecting to BBF (target)...
✅ Connected to BBF: bluebirdnetwork--full.sandbox.my.salesforce.com


In [5]:
# === QUERY ES ACCOUNTS ===
# When FILTER_BY_BBF_BAN = True:
#   Only migrate Accounts linked to BANs with BBF_Ban__c = true
#   This ensures we only migrate Accounts with qualifying Orders
# When FILTER_BY_BBF_BAN = False:
#   Migrate ALL Customer accounts (original behavior)

print("\n" + "=" * 80)
print("QUERYING ES ACCOUNTS")
print("=" * 80)

if FILTER_BY_BBF_BAN:
    # Step 1: Get Account IDs from BANs with BBF_Ban__c = true
    print("\n📌 Step 1: Getting Account IDs from BANs with BBF_Ban__c = true...")

    ban_query = """
        SELECT Account__c
        FROM Billing_Invoice__c
        WHERE BBF_Ban__c = true
          AND Account__c != null
    """

    ban_result = es_sf.query_all(ban_query)
    account_ids_from_bans = set()

    for ban in ban_result["records"]:
        if ban.get("Account__c"):
            account_ids_from_bans.add(ban["Account__c"])

    print(f"   Found {len(account_ids_from_bans)} unique Account IDs from BBF BANs")

    if len(account_ids_from_bans) == 0:
        print("\n⚠️  No BANs with BBF_Ban__c = true found!")
        print("   Run 00_uat_ban_prep.ipynb first to mark BANs for migration.")
        es_accounts = []
    else:
        # Step 2: Query those specific Accounts (not yet migrated)
        print("\n📌 Step 2: Querying Accounts not yet migrated...")

        account_ids_list = list(account_ids_from_bans)
        es_accounts = []
        chunk_size = 200  # SOQL IN clause limit

        for i in range(0, len(account_ids_list), chunk_size):
            chunk = account_ids_list[i : i + chunk_size]
            ids_str = "','".join(chunk)

            query = f"""
                SELECT Id, Name, Type, BillingStreet, BillingCity, BillingState, 
                       BillingPostalCode, BillingCountry, Phone, Website, Industry,
                       AnnualRevenue, NumberOfEmployees, Description,
                       ShippingStreet, ShippingCity, ShippingState, ShippingPostalCode,
                       ShippingCountry, AccountNumber, Site, TickerSymbol, Ownership,
                       Rating, Sic, SicDesc
                FROM Account
                WHERE Id IN ('{ids_str}')
                  AND (BBF_New_Id__c = null OR BBF_New_Id__c = '')
            """

            if TEST_MODE and len(es_accounts) >= TEST_LIMIT:
                break

            result = es_sf.query_all(query)
            es_accounts.extend(result["records"])

            if TEST_MODE and len(es_accounts) >= TEST_LIMIT:
                es_accounts = es_accounts[:TEST_LIMIT]
                break

        print(f"   Found {len(es_accounts)} Accounts to migrate (not yet migrated)")

        # Show accounts already migrated
        already_migrated = len(account_ids_from_bans) - len(es_accounts)
        if already_migrated > 0:
            print(f"   ({already_migrated} Accounts already have BBF_New_Id__c)")
else:
    # Original behavior - migrate all Customer accounts
    print("\n📌 FILTER_BY_BBF_BAN is False - querying ALL Customer accounts...")

    query = """
        SELECT Id, Name, Type, BillingStreet, BillingCity, BillingState, 
               BillingPostalCode, BillingCountry, Phone, Website, Industry,
               AnnualRevenue, NumberOfEmployees, Description,
               ShippingStreet, ShippingCity, ShippingState, ShippingPostalCode,
               ShippingCountry, AccountNumber, Site, TickerSymbol, Ownership,
               Rating, Sic, SicDesc
        FROM Account
        WHERE (BBF_New_Id__c = null OR BBF_New_Id__c = '') AND Type = 'Customer'
    """

    # Add limit for test mode
    if TEST_MODE:
        query += f" LIMIT {TEST_LIMIT}"

    print(f"Query: {query[:200]}...")
    print("\nExecuting query...")

    result = es_sf.query_all(query)
    es_accounts = result["records"]

print(f"\n✅ Found {len(es_accounts)} accounts to migrate")

if len(es_accounts) > 0:
    sample = es_accounts[0]
    print(f"\nSample Account:")
    print(f"  ID:   {sample['Id']}")
    print(f"  Name: {sample['Name']}")
    print(f"  Type: {sample.get('Type', 'N/A')}")
    print(f"  City: {sample.get('BillingCity', 'N/A')}")
elif TEST_MODE:
    print("\n⚠️  No unmigrated accounts found in test set")
else:
    print("\n✅ All accounts have been migrated!")


QUERYING ES ACCOUNTS

📌 Step 1: Getting Account IDs from BANs with BBF_Ban__c = true...
   Found 20 unique Account IDs from BBF BANs

📌 Step 2: Querying Accounts not yet migrated...
   Found 20 Accounts to migrate (not yet migrated)

✅ Found 20 accounts to migrate

Sample Account:
  ID:   0010B00001l6HcxQAE
  Name: Montage Furniture Services
  Type: Customer
  City: Grand Rapids


In [6]:
# === TRANSFORM FOR BBF ===

print("\n" + "=" * 80)
print("TRANSFORMING ACCOUNTS FOR BBF")
print("=" * 80)

bbf_accounts = []

for es_account in es_accounts:
    bbf_account = {
        # Standard fields
        "Name": es_account.get("Name"),
        "Type": es_account.get("Type"),
        "BillingStreet": es_account.get("BillingStreet"),
        "BillingCity": es_account.get("BillingCity"),
        "BillingState": es_account.get("BillingState"),
        "BillingPostalCode": es_account.get("BillingPostalCode"),
        "BillingCountry": es_account.get("BillingCountry"),
        "ShippingStreet": es_account.get("ShippingStreet"),
        "ShippingCity": es_account.get("ShippingCity"),
        "ShippingState": es_account.get("ShippingState"),
        "ShippingPostalCode": es_account.get("ShippingPostalCode"),
        "ShippingCountry": es_account.get("ShippingCountry"),
        "Phone": es_account.get("Phone"),
        "Website": es_account.get("Website"),
        "Industry": es_account.get("Industry"),
        "AnnualRevenue": es_account.get("AnnualRevenue"),
        "NumberOfEmployees": es_account.get("NumberOfEmployees"),
        "Description": es_account.get("Description"),
        "AccountNumber": es_account.get("AccountNumber"),
        "Site": es_account.get("Site"),
        "TickerSymbol": es_account.get("TickerSymbol"),
        "Ownership": es_account.get("Ownership"),
        "Rating": es_account.get("Rating"),
        "Sic": es_account.get("Sic"),
        "SicDesc": es_account.get("SicDesc"),
        # 👤 Set account owner
        "OwnerId": OWNER_ID,
        # BBF Required boolean fields (fields without write permission removed)
        # "Agent__c": False,
        # "proposalmanager__Quote_Provider__c": False,
        # "proposalmanager__Org_Account_For_PMT_Quotes__c": False,
        # "Quote_Provider__c": False,
        # "Business_Partner__c": False,
        # "Master_Agent__c": False,
        # "New_Transition__c": False,
        # "Strategic_Account__c": False,
        # "Protected_Account__c": False,
        # 🔗 Store ES Account ID for tracking
        "ES_Legacy_ID__c": es_account["Id"],
    }

    bbf_accounts.append(bbf_account)

print(f"✅ Transformed {len(bbf_accounts)} accounts")
print(
    f"   - Mapped {len([k for k in bbf_accounts[0].keys() if not k.startswith('_')])} fields per account"
)
print(f"   - Set OwnerId to {OWNER_ID}")
print(f"   - Set ES_Legacy_ID__c for tracking")


TRANSFORMING ACCOUNTS FOR BBF
✅ Transformed 20 accounts
   - Mapped 27 fields per account
   - Set OwnerId to 005Ea00000ZOGFZIA5
   - Set ES_Legacy_ID__c for tracking


In [7]:
# === INSERT TO BBF ===

print("\n" + "=" * 80)
print("INSERTING ACCOUNTS TO BBF")
print("=" * 80)

if len(bbf_accounts) == 0:
    print("⚠️  No accounts to insert")
    successful_inserts = []
    failed_inserts = []
else:
    print(f"Inserting {len(bbf_accounts)} accounts using bulk API...")
    print("(Bulk API automatically batches in 200-record chunks)\n")

    try:
        results = bbf_sf.bulk.Account.insert(bbf_accounts)

        successful_inserts = []
        failed_inserts = []

        for i, result in enumerate(results):
            if result["success"]:
                successful_inserts.append(
                    {
                        "es_id": bbf_accounts[i]["ES_Legacy_ID__c"],
                        "bbf_id": result["id"],
                        "name": bbf_accounts[i]["Name"],
                    }
                )
            else:
                failed_inserts.append(
                    {
                        "es_id": bbf_accounts[i]["ES_Legacy_ID__c"],
                        "name": bbf_accounts[i]["Name"],
                        "errors": result["errors"],
                    }
                )

        print(f"✅ Successfully inserted: {len(successful_inserts)} accounts")
        print(f"❌ Failed to insert: {len(failed_inserts)} accounts")

        if len(failed_inserts) > 0:
            print(f"\nFailed Accounts (first 5):")
            for item in failed_inserts[:5]:
                print(f"  - {item['name']} (ES ID: {item['es_id']})")
                print(f"    Errors: {item['errors']}")

        if len(successful_inserts) > 0:
            print(f"\nSample successful insert:")
            sample = successful_inserts[0]
            print(f"  ES ID:  {sample['es_id']}")
            print(f"  BBF ID: {sample['bbf_id']}")
            print(f"  Name:   {sample['name']}")

    except Exception as e:
        print(f"❌ Error during bulk insert: {e}")
        successful_inserts = []
        failed_inserts = []


INSERTING ACCOUNTS TO BBF
Inserting 20 accounts using bulk API...
(Bulk API automatically batches in 200-record chunks)

✅ Successfully inserted: 19 accounts
❌ Failed to insert: 1 accounts

Failed Accounts (first 5):
  - Windstream (ES ID: 001U000001i7MmvIAE)
    Errors: [{'statusCode': 'DUPLICATES_DETECTED', 'message': 'You are creating a duplicate record. We recommend you use an existing record instead.', 'fields': []}]

Sample successful insert:
  ES ID:  0010B00001l6HcxQAE
  BBF ID: 001Ea00001KidUyIAJ
  Name:   Montage Furniture Services


In [8]:
# === UPDATE ES WITH BBF IDS ===

print("\n" + "=" * 80)
print("UPDATING ES WITH BBF IDS")
print("=" * 80)

if len(successful_inserts) == 0:
    print("⚠️  No successful inserts to update in ES")
else:
    es_updates = []
    for item in successful_inserts:
        es_updates.append({"Id": item["es_id"], "BBF_New_Id__c": item["bbf_id"]})

    print(f"Updating {len(es_updates)} ES accounts with BBF IDs...")
    print("(Processing in batches of 10 to avoid CPQ trigger governor limits)\n")

    # Process in small batches to avoid CPQ trigger SOQL limit
    BATCH_SIZE = 10
    total_success = 0
    total_errors = 0

    try:
        for i in range(0, len(es_updates), BATCH_SIZE):
            batch = es_updates[i : i + BATCH_SIZE]
            batch_num = (i // BATCH_SIZE) + 1
            total_batches = (len(es_updates) + BATCH_SIZE - 1) // BATCH_SIZE

            print(
                f"  Processing batch {batch_num}/{total_batches} ({len(batch)} records)...",
                end=" ",
            )

            results = es_sf.bulk.Account.update(batch)

            success_count = sum(1 for r in results if r["success"])
            error_count = sum(1 for r in results if not r["success"])

            total_success += success_count
            total_errors += error_count

            if error_count > 0:
                print(f"⚠️  {success_count} OK, {error_count} failed")
                for j, result in enumerate(results):
                    if not result["success"]:
                        print(f"    Error on {batch[j]['Id']}: {result['errors']}")
            else:
                print(f"✅ {success_count} OK")

        print(f"\n✅ Successfully updated: {total_success} ES accounts")
        if total_errors > 0:
            print(f"❌ Failed to update: {total_errors} ES accounts")

        print("\n🔄 Bidirectional tracking complete:")
        print("   BBF → ES_Legacy_ID__c (original ES ID)")
        print("   ES → BBF_New_Id__c (new BBF ID)")

    except Exception as e:
        print(f"⚠️  Could not update ES accounts: {e}")
        print("(Migration can continue - ES tracking is optional)")


UPDATING ES WITH BBF IDS
Updating 19 ES accounts with BBF IDs...
(Processing in batches of 10 to avoid CPQ trigger governor limits)

  Processing batch 1/2 (10 records)... ✅ 10 OK
  Processing batch 2/2 (9 records)... ✅ 9 OK

✅ Successfully updated: 19 ES accounts

🔄 Bidirectional tracking complete:
   BBF → ES_Legacy_ID__c (original ES ID)
   ES → BBF_New_Id__c (new BBF ID)


In [9]:
# === VERIFY MIGRATION ===

print("\n" + "=" * 80)
print("VERIFYING MIGRATION")
print("=" * 80)

if len(successful_inserts) == 0:
    print("⚠️  No accounts to verify")
else:
    # Query BBF for sample of inserted accounts
    sample_ids = [item["bbf_id"] for item in successful_inserts[:10]]
    id_list = "','".join(sample_ids)

    verify_query = (
        f"SELECT Id, Name, ES_Legacy_ID__c FROM Account WHERE Id IN ('{id_list}')"
    )

    try:
        result = bbf_sf.query_all(verify_query)
        bbf_verify = result["records"]

        print(f"✅ Verified {len(bbf_verify)} accounts in BBF")
        print("\nSample verified accounts (first 3):")
        for account in bbf_verify[:3]:
            print(f"  - {account['Name']}")
            print(f"    BBF ID: {account['Id']}")
            print(f"    ES ID:  {account.get('ES_Legacy_ID__c', 'N/A')}")

    except Exception as e:
        print(f"⚠️  Could not verify: {e}")


VERIFYING MIGRATION
✅ Verified 10 accounts in BBF

Sample verified accounts (first 3):
  - Montage Furniture Services
    BBF ID: 001Ea00001KidUyIAJ
    ES ID:  0010B00001l6HcxQAE
  - D&P Communications
    BBF ID: 001Ea00001KidUzIAJ
    ES ID:  0010B00001p6Z3NQAU
  - CCI Systems
    BBF ID: 001Ea00001KidV0IAJ
    ES ID:  0014P00002cdSXiQAM


In [10]:
# === CREATE EXCEL OUTPUT ===

print("\n" + "=" * 80)
print("CREATING EXCEL OUTPUT")
print("=" * 80)

# Combine all results
all_results = []

for item in successful_inserts:
    all_results.append(
        {
            "ES_Account_Id": item["es_id"],
            "BBF_Account_Id": item["bbf_id"],
            "Account_Name": item["name"],
            "Status": "SUCCESS",
            "Error": "",
        }
    )

for item in failed_inserts:
    all_results.append(
        {
            "ES_Account_Id": item["es_id"],
            "BBF_Account_Id": "",
            "Account_Name": item["name"],
            "Status": "FAILED",
            "Error": str(item["errors"]),
        }
    )

# Identify name collisions
name_collisions = []
for item in failed_inserts:
    error_str = str(item["errors"])
    if "DUPLICATE_VALUE" in error_str and "Name" in error_str:
        name_collisions.append(
            {
                "ES_Account_Id": item["es_id"],
                "Account_Name": item["name"],
                "Error_Details": error_str,
            }
        )

print(f"Identified {len(name_collisions)} name collision errors")

# Create Excel workbook
wb = Workbook()
wb.remove(wb.active)

# Styles
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_alignment = Alignment(horizontal="center", vertical="center")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

status_colors = {"SUCCESS": "00C851", "FAILED": "FF4444"}

# --- SHEET 1: Migration Results ---
ws1 = wb.create_sheet("Migration Results")
headers1 = ["ES Account ID", "BBF Account ID", "Account Name", "Status", "Error"]
ws1.append(headers1)

for col, header in enumerate(headers1, 1):
    cell = ws1.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for row_idx, r in enumerate(all_results, 2):
    ws1.append(
        [
            r["ES_Account_Id"],
            r["BBF_Account_Id"],
            r["Account_Name"],
            r["Status"],
            r["Error"],
        ]
    )
    fill_color = status_colors.get(r["Status"], "FFFFFF")
    for col in range(1, len(headers1) + 1):
        cell = ws1.cell(row=row_idx, column=col)
        cell.fill = PatternFill("solid", fgColor=fill_color)
        cell.border = thin_border

for col in ws1.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws1.column_dimensions[col[0].column_letter].width = min(max_length + 2, 60)

ws1.freeze_panes = "A2"

# --- SHEET 2: Summary ---
ws2 = wb.create_sheet("Summary")
ws2.append(["ES → BBF Account Migration Summary"])
ws2["A1"].font = Font(bold=True, size=14)
ws2.append([])
ws2.append(["Run Type:", "TEST MODE" if TEST_MODE else "FULL MIGRATION"])
ws2.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws2.append(["Owner ID:", OWNER_ID])
ws2.append([])
ws2.append(["Metric", "Count"])
ws2["A7"].font = Font(bold=True)
ws2["B7"].font = Font(bold=True)
ws2.append(["Total Accounts Processed", len(all_results)])
ws2.append(["Successful Inserts", len(successful_inserts)])
ws2.append(["Failed Inserts", len(failed_inserts)])
ws2.append(["Name Collisions", len(name_collisions)])
ws2.append(
    [
        "Success Rate",
        (
            f"{len(successful_inserts)/len(all_results)*100:.1f}%"
            if len(all_results) > 0
            else "0%"
        ),
    ]
)

# --- SHEET 3: ID Mapping ---
ws3 = wb.create_sheet("ID Mapping")
headers3 = ["ES Account ID", "BBF Account ID", "Account Name"]
ws3.append(headers3)

for col, header in enumerate(headers3, 1):
    cell = ws3.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for item in successful_inserts:
    ws3.append([item["es_id"], item["bbf_id"], item["name"]])

for col in ws3.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws3.column_dimensions[col[0].column_letter].width = min(max_length + 2, 50)

ws3.freeze_panes = "A2"

# --- SHEET 4: Name Collisions ---
ws4 = wb.create_sheet("Name Collisions")
headers4 = ["ES Account ID", "Account Name (Duplicate)", "Error Details"]
ws4.append(headers4)

for col, header in enumerate(headers4, 1):
    cell = ws4.cell(row=1, column=col)
    cell.font = Font(bold=True, size=12, color="FFFFFF")
    cell.fill = PatternFill(start_color="FF4444", end_color="FF4444", fill_type="solid")
    cell.alignment = header_alignment

for collision in name_collisions:
    ws4.append(
        [
            collision["ES_Account_Id"],
            collision["Account_Name"],
            collision["Error_Details"],
        ]
    )

for col in ws4.columns:
    max_length = max(len(str(cell.value)) for cell in col) if list(col) else 10
    ws4.column_dimensions[col[0].column_letter].width = min(max_length + 2, 70)

ws4.freeze_panes = "A2"

# Save workbook
wb.save(output_file)
print(f"\n✅ Excel output saved to: {output_file}")
print(f"   📊 Sheet 1: Migration Results ({len(all_results)} accounts, color-coded)")
print(f"   📈 Sheet 2: Summary (metrics and stats)")
print(f"   🔗 Sheet 3: ID Mapping ({len(successful_inserts)} successful mappings)")
print(f"   ⚠️  Sheet 4: Name Collisions ({len(name_collisions)} duplicate name errors)")


CREATING EXCEL OUTPUT
Identified 0 name collision errors

✅ Excel output saved to: es_bbf_account_migration_20260114_095529.xlsx
   📊 Sheet 1: Migration Results (20 accounts, color-coded)
   📈 Sheet 2: Summary (metrics and stats)
   🔗 Sheet 3: ID Mapping (19 successful mappings)
   ⚠️  Sheet 4: Name Collisions (0 duplicate name errors)


In [11]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("MIGRATION COMPLETE")
print("=" * 80)
print(f"ES Accounts queried: {len(es_accounts)}")
print(f"BBF Accounts inserted: {len(successful_inserts)}")
print(
    f"Success rate: {len(successful_inserts)/len(es_accounts)*100:.1f}%"
    if len(es_accounts) > 0
    else "0%"
)
print(f"\nExcel output: {output_file}")

if TEST_MODE:
    print("\n🔄 TEST MODE complete. Only migrated " + str(TEST_LIMIT) + " accounts.")
    print("   To migrate ALL accounts, set TEST_MODE = False in Cell 2 and re-run.")
else:
    print("\n✅ FULL MIGRATION complete!")
    print("   Ready for child object migrations.")
    print("   Next: Migrate Contacts, BANs, Opportunities using ID Mapping sheet")

if len(name_collisions) > 0:
    print(f"\n⚠️  {len(name_collisions)} accounts failed due to duplicate names")
    print("   Check 'Name Collisions' sheet in Excel")
    print("   Rename in ES and re-run (already-migrated accounts will be skipped)")


MIGRATION COMPLETE
ES Accounts queried: 20
BBF Accounts inserted: 19
Success rate: 95.0%

Excel output: es_bbf_account_migration_20260114_095529.xlsx

✅ FULL MIGRATION complete!
   Ready for child object migrations.
   Next: Migrate Contacts, BANs, Opportunities using ID Mapping sheet


---
## Next Steps: Child Object Migration

After Account migration is complete, use the **ID Mapping sheet** from the Excel output to migrate child objects:

1. **Contact** (needs Account ID)
2. **BAN__c** (needs Account ID)
3. **BAN_Contact__c** (needs BAN + Contact IDs)
4. **Opportunity** (needs Account + Contact IDs)
5. **Opportunity_Site__c** (needs Opportunity ID)

See `migration_script_template.py` for child object migration examples.

---
## Package Installation

If you need to install packages, run this cell:

In [ ]:
# Install required packages
!pip install simple-salesforce pandas openpyxl

## Delete Migrated Accounts from BBF

List<Account>a = [SELECT Id, Name FROM Account WHERE Owner.Name = 'Everstream Legacy'];
System.debug(a);
delete a;

## remove BBF_New_Id__c for moved orders
List<Account> accountIds = [SELECT Id, BBF_New_Id__c FROM Account WHERE BBF_New_Id__c != NULL OR BBF_New_Id__c !=''];
System.debug(accountIds);
for (Account a : accountIds){
    a.BBF_New_Id__c = NULL;
}
update accountIds;